### Business Problem

Fetch Rewards is a mobile app where you can earn free gift cards by scanning and uploading your shopping receipts. You accumulate points for eligible receipts, which can be redeemed for various gift cards. It's a way to get rewards for your everyday shopping.

Fetch provides value to their user base through the rich variety of offers that are active in the app. They want their users to be able to easily seek out offers in the app, so that they get the most out of using the app and their partners get the most out of their relationship with Fetch.

### Solution

Build a tool that allows users to intelligently search for offers via text input from the user.

#### Functional requirements : 

- If a user searches for a category (ex. diapers) the tool should return a list of offers that are relevant to that category.
- If a user searches for a brand (ex. Huggies) the tool should return a list of offers that are relevant to that brand.
- If a user searches for a retailer (ex. Target) the tool should return a list of offers that are relevant to that retailer.
- The tool should also return the score that was used to measure the similarity of the text input with each offer

#### Non-functional requirements : 
- Needs to have low latency. The search results should be ideally displayed instantaneously i.e. in less than a couple of seconds. 

### Machine Learning Problem Fromulation

An information retrieval system is required such that when a word is given as input, it searches the entire database and returns the text that is syntactically similar to the search query. As we are searching through database using a name of brand or retailer (or category), there isn't much scope for searching based on semantic meaning (because 'names' don't really carry any meaning). So, the search is more likely to be focused on text similarity or syntactic similarity. 

Put in simple terms, if we had a data that is in the form of story or kind and the search query would have been something one the lines of 'why did he go to XYZ', 'how did she achieve it?', etc where the questions have an actual meaning and so we can use semantic search and Language models to find answers. But, in this case, the query is just a word which is a 'name' and therefore don't really have any semantic meaning. For example, the word 'Huggies' is just a brand name and it does not mean something. 

### How can it be solved ? 

#### 1. Rule based approach : 
Use regular expressions to search for the given search query (like brand name, category or retailer) in the database. 

##### Pros : 
a. Will work well for all the offers that have the search query in them. <br>
b. Will fetch exactly those records that have the search query term in them. <br>

##### Cons : 
a. It will be extremely complicated to code as lot of if-else conditions will need to be included. <br>
b. The time complexity of this would be O(n * m) where n is the number of offers and m is the number of words in the offer. <br>
c. It will NOT be robust, a small spelling mistake or change in text will lead to failure of the system. <br>

#### 2. Using sentence embeddings and finding similarity therein : 
Use sentence embeddings to vectorize all the data i.e. list of all the offers. Use cosine similarity to find the similarity between the offer and search query. Sort the results thus obtained using similarity score. Display the records with highest similarity scores. It is the required output. 

###### Pros : 
a. It is a robust approach, a spelling mistake here and there won't drastically affect the system. <br>
b. It will be of O(n log n) time complexity. As we will just need to compute the similarity scores of all records i.e. O(n) and then sort them according to similarity score i.e. O(n log n). So relatively, it will be faster than the rule based approach. We can further bring down the time complexity and make it even more faster by using a vector database like Redis.<br> 
c. Much simplified code unlike the rule based approach. <br>

##### Cons : 
a. If the offer text contains a word that is syntactically (textually similar) to some brand name or category or retailer name, then that offer too will be displayed i.e. a False positive is likely to occur in the search result.

### Solution Approach:

1. We have a dataset that has 'offer details', 'brand name', 'category name', 'retailer name'. One offer can be associated with brand, a retailer and have certain category all at the same time or might have one or two details missing. We need to provide the details of offer even if anyone of those three fields is given as input. However, it is not necessary that the brand name or category would certainly be mentioned in the offer details. So, a smart hack around this would be to concatenate the offer details with available metadata i.e. 'brand name', 'category name' and 'retailer name'. 

2. The next step would be to convert all those strings/texts/documents into vectors so that we can easily find the cosine similarity between the search query and the offer. We can store these vectors in a file (for extremely small datasets and toy datasets like this one) or use a vector database (like the one provided by Redis). 

3. Once we have all the dataset in vector form, at run time, we just need to vectorize the input search query and compute the cosine similarity between the search query and embeddings in the vector database. Post that we can arrange them in desceding order of similarity score and display the top-10 results to the user.

### How to scale it for production/industry level deployment ?

In order to deploy this system in production, a sophisticated database like Redis, which is a vector database should be used. Further, LangChain can be used to retrieve similar records from the vector database while maintaing the low latency for real-time application.